# NASA INSIGHT API

In [1]:
import requests
import json
import pandas as pd

In [2]:
from datetime import datetime
datetime.now()

datetime.datetime(2024, 5, 20, 10, 33, 32, 669860)

In [3]:
with open('../config.json', 'r') as jsonfile:
    data = json.load(jsonfile)
api_key = data['api_key']

In [5]:
url = f'https://api.nasa.gov/insight_weather/?api_key={api_key}&feedtype=json'

In [4]:
def get_mars_weather():
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        print(data)

        sol_keys = data['sol_keys']
        print(f"Latest available Sols: {sol_keys}")

        for sol in sol_keys:
            sol_data = data[sol]
            print(f"\nWeather for Sol {sol}:")
            if 'AT' in sol_data:
                print(f"Average Temperature: {sol_data['AT']['av']}°C")
            else:
                print("Temperature data is not available.")
            
            if 'HWS' in sol_data:
                print(f"Average Wind Speed: {sol_data['HWS']['av']} m/s")
            else:
                print("Wind speed data is not available.")
            
            if 'PRE' in sol_data:
                print(f"Average Atmospheric Pressure: {sol_data['PRE']['av']} Pa")
            else:
                print("Pressure data is not available.")
            
            if 'WD' in sol_data and sol_data['WD'].get('most_common'):
                print(f"Most Common Wind Direction: {sol_data['WD']['most_common']['compass_point']}")
            else:
                print("Wind direction data is not available.")

            print(f"Mars Season: {sol_data['Season']}")
            print(f"First Data UTC: {sol_data['First_UTC']}")
            print(f"Last Data UTC: {sol_data['Last_UTC']}")

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except KeyError as e:
        print(f"Error processing data: {e} - The structure of the data might have changed or it's currently unavailable.")

In [5]:
get_mars_weather()

{'sol_keys': [], 'validity_checks': {'1219': {'AT': {'sol_hours_with_data': [6, 7, 8, 9, 10, 11, 12], 'valid': False}, 'HWS': {'sol_hours_with_data': [6, 7, 8, 9, 10, 11, 12], 'valid': False}, 'PRE': {'sol_hours_with_data': [6, 7, 8, 9, 10, 11, 12], 'valid': False}, 'WD': {'sol_hours_with_data': [6, 7, 8, 9, 10, 11, 12], 'valid': False}}, 'sol_hours_required': 18, 'sols_checked': ['1219']}}
Latest available Sols: []


## Solar flare

In [7]:
mars_data = pd.read_csv('../data/mars-weather.csv')
mars_data.terrestrial_date.min(), mars_data.terrestrial_date.max()

('2012-08-07', '2018-02-27')

In [10]:
mars_data['terrestrial_date'] = pd.to_datetime(mars_data['terrestrial_date'])

In [11]:
mars_data2 = pd.read_csv('../data/marsWeather.csv')
mars_data2['terrestrial_date'] = pd.to_datetime(mars_data2['terrestrial_date'])
mars_data2.terrestrial_date.min(), mars_data2.terrestrial_date.max()

(Timestamp('2014-01-02 00:00:00'), Timestamp('2022-11-25 00:00:00'))

In [13]:
start_date = mars_data.terrestrial_date.min().strftime('%Y-%m-%d')
end_date = mars_data2.terrestrial_date.max().strftime('%Y-%m-%d')

current_date = datetime.strptime(start_date, '%Y-%m-%d')
end_date = datetime.strptime(end_date, '%Y-%m-%d')

In [14]:
from datetime import datetime, timedelta

while current_date <= end_date:
    next_month_date = current_date + timedelta(days=32)  # Move to the next month
    next_month_date = next_month_date.replace(day=1)  # Set day to the first of the next month

    # Format dates for the API
    start_month = current_date.strftime('%Y-%m-%d')
    end_month = (next_month_date - timedelta(days=1)).strftime('%Y-%m-%d')

    # Create the API URL
    solar_flare_url = f'https://api.nasa.gov/DONKI/FLR?startDate={start_month}&endDate={end_month}&api_key={api_key}'

    try:
        response = requests.get(solar_flare_url)
        response.raise_for_status()
        data = response.json()

        # Normalize and save data if any flares were captured
        if data:
            normalized_data = pd.json_normalize(data)
            file_name = f'../data/APIdata/solar_data_{start_month}_to_{end_month}.csv'
            normalized_data.to_csv(file_name)
            print(f'Data saved for {start_month} to {end_month}')

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")

    # Move to the next month
    current_date = next_month_date

Data saved for 2012-08-07 to 2012-08-31
Data saved for 2012-09-01 to 2012-09-30
Data saved for 2012-10-01 to 2012-10-31
Data saved for 2012-11-01 to 2012-11-30
Data saved for 2013-03-01 to 2013-03-31
Data saved for 2013-04-01 to 2013-04-30
Data saved for 2013-05-01 to 2013-05-31
Data saved for 2013-06-01 to 2013-06-30
Data saved for 2013-10-01 to 2013-10-31
Data saved for 2013-11-01 to 2013-11-30
Data saved for 2013-12-01 to 2013-12-31
Data saved for 2014-01-01 to 2014-01-31
Data saved for 2014-02-01 to 2014-02-28
Data saved for 2014-03-01 to 2014-03-31
Data saved for 2014-04-01 to 2014-04-30
Data saved for 2014-05-01 to 2014-05-31
Data saved for 2014-06-01 to 2014-06-30
Data saved for 2014-07-01 to 2014-07-31
Data saved for 2014-08-01 to 2014-08-31
Data saved for 2014-09-01 to 2014-09-30
Data saved for 2014-10-01 to 2014-10-31
Data saved for 2014-11-01 to 2014-11-30
Data saved for 2014-12-01 to 2014-12-31
Data saved for 2015-01-01 to 2015-01-31
Data saved for 2015-02-01 to 2015-02-28


In [16]:
# we can change the dates 
solar_flare_url = f'https://api.nasa.gov/DONKI/FLR?startDate=2024-03-01&endDate=2024-04-01&api_key={api_key}'

how it looks like

```python
[{"flrID":"2016-01-01T23:00:00-FLR-001","instruments":[{"displayName":"GOES15: SEM/XRS 1.0-8.0"}],"beginTime":"2016-01-01T23:00Z","peakTime":"2015-01-02T00:10Z","endTime":null,"classType":"M2.3","sourceLocation":"S21W73","activeRegionNum":12473,"linkedEvents":[{"activityID":"2016-01-01T23:12:00-CME-001"},{"activityID":"2016-01-02T02:48:00-SEP-001"},{"activityID":"2016-01-02T04:30:00-SEP-001"}],"link":"https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/FLR/9963/-1"},{"flrID":"2016-01-28T11:48:00-FLR-001","instruments":[{"displayName":"GOES15: SEM/XRS 1.0-8.0"}],"beginTime":"2016-01-28T11:48Z","peakTime":"2016-01-28T12:02Z","endTime":"2016-01-28T12:56Z","classType":"C9.6","sourceLocation":"N03W47","activeRegionNum":12488,"linkedEvents":[{"activityID":"2016-01-28T12:24:00-CME-001"}],"link":"https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/FLR/10122/-1"}]
```

In [17]:
try:
    response = requests.get(solar_flare_url)
    response.raise_for_status()
    data = response.json()
    print(len(data))
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

1


In [13]:
data[0]

{'flrID': '2024-03-07T22:02:00-FLR-001',
 'instruments': [{'displayName': 'GOES-P: EXIS 1.0-8.0'}],
 'beginTime': '2024-03-07T22:02Z',
 'peakTime': '2024-03-07T22:33Z',
 'endTime': '2024-03-07T23:24Z',
 'classType': 'C3.5',
 'sourceLocation': 'N17W90',
 'activeRegionNum': None,
 'note': 'Brightening is seen in SDO AIA 131 on or just beyond the NW limb at about N17. This brightening simultaneously occurs with opening field lines and ejecta which are also observed just beyond the NW limb at abut N17 in SDO AIA 131 starting at 2024-03-07T22:09Z.',
 'linkedEvents': [{'activityID': '2024-03-07T22:36:00-CME-001'}],
 'submissionTime': '2024-03-08T19:01Z',
 'link': 'https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/FLR/29506/-1'}

In [18]:
normalized_data = pd.json_normalize(data)
normalized_data

,flrID,instruments,beginTime,peakTime,endTime,classType,sourceLocation,activeRegionNum,note,linkedEvents,submissionTime,link
0,2024-04-01T01:13:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-04-01T01:13Z,2024-04-01T01:32Z,2024-04-01T01:57Z,M3.9,S13W108,13615,There is also a simultaneous sympathetic erupt...,None,2024-04-01T02:05Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...


In [19]:
normalized_data.to_csv('../data/APIdata/solar_data_2024-04-01_to_2024-04-03.csv')

https://www.swpc.noaa.gov/phenomena/solar-flares-radio-blackouts

Features:

- **flrID**: This is an identifier for the flare event.
- **instruments**: These are the tools or instruments used to observe the flare (e.g., GOES15: SEM/XRS 1.0-8.0).
- **beginTime**, peakTime, endTime: These represent the start time, peak time, and end time of the flare event.
- **sourceLocation**: This indicates the solar coordinates (latitude and longitude on the solar surface) where the flare originated.
- **activeRegionNum**: This is the NOAA active region number, which is a unique identifier for the region of the sun where the flare occurred.
- **linkedEvents**: This includes IDs of other space weather events linked to the flare, such as Coronal Mass Ejections (CMEs) or Solar Energetic Particle (SEP) events.
- **link**: This provides a URL to more information about the specific flare event.

NOAA Space Weather Scales

- X-Class Flares: These are the most intense flares. They are major events that can trigger planet-wide radio blackouts and long-lasting radiation storms.

- M-Class Flares: These are medium-sized; they can cause brief radio blackouts that affect Earth's polar regions. Minor radiation storms might occur.

- C-Class Flares: These are small with few noticeable consequences on Earth, except for minor disruptions in polar radio communications.

- B-Class Flares: These are even smaller, with little to no effect on Earth.

- A-Class Flares: These are the smallest flares, which are similar to B-Class in their lack of significant impact on Earth.


The number that follows the letter represents the peak flux (in watts per square meter, W/m²) of x-rays near Earth, as measured in the 1 to 8 angstrom range. For example, an M2.3 class flare has a peak flux between 2x10^-5 and 3x10^-5 W/m².

